<a href="https://colab.research.google.com/github/1028Luo/LLM-Domain-Specific-Assistant/blob/main/RAG_LLAMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install llama-index
!pip install llama-index-embeddings-huggingface
!pip install -q bitsandbytes==0.45.0
!pip install -q transformers==4.47.1
!pip install -q huggingface_hub
!pip install numba

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 9.2 MB/s eta 0:00:00


In [2]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from numba import cuda
import torch
import bitsandbytes

In [3]:
# Log in to Hugging Face
from google.colab import userdata
from huggingface_hub import login
my_hugging_face_token = userdata.get('huggingface_token')
login(token=my_hugging_face_token)

In [4]:
# load local documents
documents = SimpleDirectoryReader("drive/MyDrive/Colab Notebooks/RAG_data").load_data()
print("length of document: ", len(documents))

# import any embedding model on HF hub (https://huggingface.co/spaces/mteb/leaderboard)
# embedding models are used to convert text (query, doc) into embedding vectors
# they are usually transformers too
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

# parameters
Settings.llm = None
Settings.chunk_size = 512
Settings.chunk_overlap = 25

# store docs in embedding space using the embedding model
index = VectorStoreIndex.from_documents(documents)

# set number of docs to retreive
top_k = 6

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=top_k,
)

# query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.5)],
)


length of document:  37


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

LLM is explicitly disabled. Using MockLLM.


In [5]:
# load model and tokenizer
# model_name = "Jiexing1028/llama-3-1b-miniguanaco-Luo"
model_name = "meta-llama/Llama-3.2-1B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                            device_map="cuda",
                                            trust_remote_code=False,
                                            revision="main")
 # use a different tokenizer for now because the pretrained model does not have chat template
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct", use_fast=True)


config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [6]:
def chat():
    while True:
        question = input("You: ")
        if question.lower() == 'exit':
            break
        # query documents

        response = query_engine.query(question)

        # reformat response
        context = "Context:\n"
        for i in range(top_k):
            context = context + response.source_nodes[i].text + "\n\n"
            # # prompt with no context
        # intstructions_string = f"""You are called MyGPT, an expert assistant in Robotics and AI, providing detailed and accurate responses to any query about the knowledge that you have learnt and  in your RAG data base.\
        # MyGPT will tailor the length of its responses to match the query, providing clear explanation on papers, projects and concepts in the field of robotics and AI \
        # and should keep the answer concise.

        # Please respond to the following question.
        # """
        # prompt_template = lambda comment: f'''[INST] {intstructions_string} \n{comment} \n[/INST]'''

        # prompt = prompt_template(question)
        # # print(prompt)

        # model.eval()

        # inputs = tokenizer(prompt, return_tensors="pt")
        # outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280)

        # print(tokenizer.batch_decode(outputs)[0])


        # ##### with Context #####
        # prompt_template_w_context = lambda context, question: f"""[INST]You are called MyGPT, an expert assistant in Robotics and AI, providing detailed and accurate responses to any query about the knowledge that you have learnt and  in your RAG data base.\
        # MyGPT will tailor the length of its responses to match the query, providing clear explanation on papers, projects and concepts in the field of robotics and AI \
        # and should keep the answer concise.

        # {context}
        # Please respond to the following question. Use the context above if it is helpful.

        # {question}
        # [/INST]
        # """


        prompt = [
            {"role": "system", "content": "You are called MyGPT, an expert assistant in Robotics and AI, providing detailed and accurate responses to any query about the knowledge that you have learnt and  in your RAG data base.\
        MyGPT will tailor the length of its responses to match the query, providing clear explanation on papers, projects and concepts in the field of robotics and AI \
        and should keep the answer concise. You can use the provided context if needed."},
            {"role": "system", "content": context},
            {"role": "user", "content": question},
        ]

        pipe = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
        )

        generation_args = {
            "max_new_tokens": 500,
            "return_full_text": False,
            "temperature": 0.0,
            "do_sample": False,
            "pad_token_id": tokenizer.pad_token_id
        }
        print('aaa')
        output = pipe(prompt, **generation_args)
        print(output[0]['generated_text'])


In [7]:
chat()

You: hello


Device set to use cuda
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


aaa
Hello! How can I assist you today?
You: tell me about openVLA


Device set to use cuda
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


aaa
OpenVLA (Open-Source VLA) is a modular PyTorch codebase for training VLA (Visual-Action-Learning) models. Here's an overview:

**What is VLA?**

VLA is a type of deep learning model that combines vision, action, and learning capabilities. It's designed to learn from data in a variety of tasks, such as object detection, tracking, and manipulation.

**Key Components:**

1. **Vision Encoder:** A pre-trained vision encoder, such as SigLIP (Segmented Image with Object Layers), is used to extract features from images.
2. **Action Encoder:** A pre-trained action encoder, such as DinoV2 (Deep Dense Network for Visual Action Learning), is used to extract features from actions.
3. **Policy Network:** A policy network, such as Llama 2, is used to learn the mapping between actions and the corresponding visual features.
4. **Reward Function:** A reward function, such as the VLA Reward Function, is used to evaluate the performance of the model.

**OpenVLA Architecture:**

The OpenVLA architectur

Device set to use cuda
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


aaa
The development of OpenVLA is attributed to a team at Stanford University, led by Moo Jin Kim, Karl Pertsch, Siddharth Karamcheti, Ted Xiao, Ashwin Balakrishna, Suraj Nair, Rafael Rafailov, Ethan Foster, Grace Lam, Pannag Sanketi, Quan Vuong, Thomas Kollar, Benjamin Burchfiel, Russ Tedrake, and Dorsa Sadigh.


KeyboardInterrupt: Interrupted by user

In [8]:
# free VRAM
device = cuda.get_current_device()
device.reset()
print('finished')

finished


In [ ]:
# 3.2 3B instruct good
# 3.2 1B my fine tune bad
# 3.2 1B instruct good but not as good as 3B
# need to better fine tune